In [ ]:
import torch
import torch.nn as nn
from torch.func import vmap, grad, functional_call  # Nouvelle API PyTorch 2.x

# Modèle simple
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(50, 1)

    def forward(self, x):
        return self.linear(x)

# Initialisation
model = SimpleModel()
criterion = nn.MSELoss()

# Génération de données
batch_size = 5
x = torch.randn(batch_size, 50)  # Pas besoin de requires_grad=True ici
y = torch.randn(batch_size, 1)

# Fonction qui retourne la loss pour un seul échantillon
def compute_loss(params, buffers, x_sample, y_sample):
    output = functional_call(model, params, (x_sample.unsqueeze(0),))  # Passage avant avec params donnés
    return criterion(output, y_sample.unsqueeze(0))  # Perte

# Obtenir les paramètres du modèle sous forme de dictionnaire
params = {name: param for name, param in model.named_parameters()}
buffers = {name: buffer for name, buffer in model.named_buffers()}  # Pour les modules comme BatchNorm

# Calcul du gradient de la loss par rapport aux paramètres
compute_grad = grad(compute_loss)

# Vectoriser pour tout le batch
batched_grads = vmap(compute_grad, (None, None, 0, 0))(params, buffers, x, y)

# Calcul de la variance des gradients
gradient_variance = {name: torch.var(batched_grads[name], dim=0)
                     for name in batched_grads.keys()}

# Calcul de la moyenne des gradients
gradient_mean = {name: torch.mean(batched_grads[name], dim=0)
                     for name in batched_grads.keys()}

# Affichage
p = 0.8
for name, var in gradient_variance.items():
    mean = gradient_mean[name]
    print(f"Seuils gradient pour {name}: {torch.sum(torch.distributions.Normal(0, 1).cdf(torch.abs(mean) / var) > p)}")

Seuils gradient pour linear.weight: 1
Seuils gradient pour linear.bias: 0
